In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights

device = torch.device("cuda")

weights = EfficientNet_B4_Weights.DEFAULT
model = efficientnet_b4(weights=weights)

# 이진 분류 - 모델 가져옴
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)#nn.linear(입력텐서, 출력텐서)
model = model.to(device)


In [ ]:
#transforms 전처리
train_transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=weights.transforms().mean,
                         std=weights.transforms().std)
])

val_transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize(mean=weights.transforms().mean,
                         std=weights.transforms().std)
])


In [ ]:
# 모델의 특징 추출 부분을 고정
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
# 설정
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.amp.GradScaler("cuda")


In [ ]:
# 학습 루프
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.unsqueeze(1).float().to(device)#BCEWithLogitsLoss는 (N, 1) 형태의 레이블을 기대하므로, unsqueeze로 차원을 추가하고 float로 변환

        optimizer.zero_grad()#기울기 초기화 - torch는 기울기 누적시스템

        with torch.amp.autocast("cuda"):#자동 혼합 정밀도 사용
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()#기울기 계산
        scaler.step(optimizer)#가중치 업데이트
        scaler.update()

        total_loss += loss.item()

    return total_loss / len(loader)


In [6]:
def validate(model, loader):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.unsqueeze(1).float().to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()

    return total_loss / len(loader)


In [ ]:
import torch.nn.functional as F

def tta_predict(model, loader):
    model.eval()
    preds = []

    with torch.no_grad():#한개에 이미지에 2가지 예측을 평균내서 최종 예측값
        for images in loader:
            images = images.to(device)

            # 원본
            outputs1 = F.sigmoid(model(images))

            # 좌우 반전
            flipped = torch.flip(images, dims=[3])
            outputs2 = F.sigmoid(model(flipped))

            # 평균
            outputs = (outputs1 + outputs2) / 2
            preds.extend(outputs.cpu().numpy())#preds 리스트에 예측값 추가 - cpu로 이동

    return preds


In [8]:
import os
import pandas as pd
filenames = os.listdir('train')
labels = ['dog' if 'dog' in name else 'cat' for name in filenames]
df = pd.DataFrame({'filename' : filenames, 'label' : labels})
print(df.head())

        filename label
0   cat.9920.jpg   cat
1  cat.11403.jpg   cat
2   cat.2180.jpg   cat
3   cat.1109.jpg   cat
4    dog.696.jpg   dog


In [9]:
# Make custom dataset // 이미지텐서, 라벨텐서로 변환하는 과정
from torch.utils.data import Dataset, DataLoader
from PIL import Image
class Dataframedataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = 1 if self.dataframe.iloc[idx, 1] == 'dog' else 0
        label = torch.tensor(label, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

In [10]:
# split Dataset
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42, 
    stratify=df['label']
    )

In [11]:
train_dataset = Dataframedataset(
    dataframe=train_df,
    img_dir='train',
    transform=train_transform
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = Dataframedataset(
    dataframe=val_df,
    img_dir='train',
    transform=val_transform
)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [12]:
for epoch in range(20):
    train_loss = train_one_epoch(model, train_loader)
    val_loss = validate(model, val_loader)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")


Epoch 1: Train Loss=0.5688, Val Loss=0.4644
Epoch 2: Train Loss=0.3997, Val Loss=0.3367
Epoch 3: Train Loss=0.3079, Val Loss=0.2659
Epoch 4: Train Loss=0.2515, Val Loss=0.2174
Epoch 5: Train Loss=0.2131, Val Loss=0.1854
Epoch 6: Train Loss=0.1860, Val Loss=0.1624
Epoch 7: Train Loss=0.1665, Val Loss=0.1398
Epoch 8: Train Loss=0.1522, Val Loss=0.1284
Epoch 9: Train Loss=0.1389, Val Loss=0.1138
Epoch 10: Train Loss=0.1289, Val Loss=0.1065
Epoch 11: Train Loss=0.1219, Val Loss=0.0976
Epoch 12: Train Loss=0.1149, Val Loss=0.0902
Epoch 13: Train Loss=0.1080, Val Loss=0.0861
Epoch 14: Train Loss=0.1020, Val Loss=0.0825
Epoch 15: Train Loss=0.0979, Val Loss=0.0755
Epoch 16: Train Loss=0.0954, Val Loss=0.0718
Epoch 17: Train Loss=0.0947, Val Loss=0.0706
Epoch 18: Train Loss=0.0879, Val Loss=0.0656
Epoch 19: Train Loss=0.0873, Val Loss=0.0643
Epoch 20: Train Loss=0.0848, Val Loss=0.0623


In [13]:
class TestDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_dir = img_dir
        self.transform = transform
        self.filenames = os.listdir(img_dir)

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.filenames[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image
test_dataset = TestDataset(
    img_dir='test',
    transform=val_transform
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [17]:
import numpy as np


test_preds = tta_predict(model, test_loader)
test_preds = np.array(test_preds).reshape(-1)

submission = pd.DataFrame({
    "id": test_dataset.filenames,
    "label": test_preds
})

submission["id"] = submission["id"].str.replace(".jpg", "", regex=False)

submission.to_csv("submission.csv", index=False)
